<a href="https://colab.research.google.com/github/akshayugalmogale/Neural-Networks-and-Deep-Learning-Practice/blob/main/Keras_Optimize_CNN_Model_DataFashion_Mnist_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import keras_tuner
from tensorflow import keras
from keras_tuner.engine.hyperparameters import HyperParameters

In [4]:
fashion_mnist=keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
from tensorflow.python import metrics
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int("conv_1_filter",min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
          ),
      keras.layers.Conv2D(
          filters=hp.Int("conv_2_filter",min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
          ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation="relu"
          ),
      keras.layers.Dense(10,activation="softmax")
  
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate",values=[1e-2,1e-3])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [9]:
tuner_search = keras_tuner.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='output',
    project_name="FashionMnist")

In [10]:
tuner_search.search(train_images,train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 27s]
val_accuracy: 0.1054999977350235

Best val_accuracy So Far: 0.9104999899864197
Total elapsed time: 00h 03m 14s


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        76864     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 96)                2973792   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3,052,106
Trainable params: 3,052,106
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1301 - accuracy: 0.9513 - val_loss: 0.2840 - val_accuracy: 0.9170
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0873 - accuracy: 0.9672 - val_loss: 0.2869 - val_accuracy: 0.9162
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0630 - accuracy: 0.9774 - val_loss: 0.3443 - val_accuracy: 0.9175
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0464 - accuracy: 0.9828 - val_loss: 0.4181 - val_accuracy: 0.9177
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0357 - accuracy: 0.9869 - val_loss: 0.4102 - val_accuracy: 0.9138
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0297 - accuracy: 0.9895 - val_loss: 0.4585 - val_accuracy: 0.9137
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0241 - accuracy: 0.9919 - val_loss: 0.5795 - val_accuracy